In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv
import re
from tqdm import tqdm

In [2]:
urls = ["https://donut.caltech.edu/1/users?page=1&name=&house_id=&option_id=&building_id=&grad_year=2023&username=&email=&timezone_from=&timezone_to=&total=287&per_page=287",
        "https://donut.caltech.edu/1/users?page=1&name=&house_id=&option_id=&building_id=&grad_year=2024&username=&email=&timezone_from=&timezone_to=&total=253&per_page=253",
        "https://donut.caltech.edu/1/users?page=1&name=&house_id=&option_id=&building_id=&grad_year=2025&username=&email=&timezone_from=&timezone_to=&total=281&per_page=281",
        "https://donut.caltech.edu/1/users?page=1&name=&house_id=&option_id=&building_id=&grad_year=2026&username=&email=&timezone_from=&timezone_to=&total=238&per_page=238"]

user_refs_year = {}
for url in urls:
    page_year = requests.get(url)
    soup_year = BeautifulSoup(page_year.content, 'html.parser')
    for anchor in soup_year.find_all("a"): 
        # extract linked url from the anchor
        if "href" in anchor.attrs:
            link = anchor.attrs["href"]
            if "/1/users" in link and "timezone" not in link:
                user_refs_year[anchor] = [link]
        else:
            link = ''
            user_refs_year[anchor] = [link]

In [21]:
emails = []
for i in tqdm(user_refs_year.keys()):
    page1 = requests.get(f'https://donut.caltech.edu{user_refs_year[i][0]}')
    soup1 = BeautifulSoup(page1.content, 'html.parser')
    refs = soup1.find_all("a")
    label = soup1.find_all("h2")
    for lab in label:
        name = lab.text.replace('\n', '').replace('\t', '')
    email = ''
    for anchor in refs:
        if "mailto" in anchor.attrs["href"]:
            email = anchor.attrs["href"].replace('mailto:', '')
    emails.append([name, email])

100%|██████████████████████████████████████████████████████████████████████████████| 1061/1061 [02:27<00:00,  7.19it/s]


In [23]:
with open('ug_emails.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for i in range(len(emails)):
        if ' ' not in emails[i][0]:
            m = re.search(r'^([^A-Z]*[A-Z]){2}', emails[i][0]);
            emails[i][0] = emails[i][0][0:m.span()[1] - 1] + ' ' + emails[i][0][m.span()[1] - 1:]
        writer.writerow([emails[i][0], emails[i][1]])

In [24]:
print(emails)

[['Max Adang', 'madang@caltech.edu'], ['Olivia Addington', 'oliviaa@caltech.edu'], ['Gabriel Aguiar', 'gpa@caltech.edu'], ['Jedi Alindogan', 'jalindog@caltech.edu'], ['Eric Amaro', 'eamaro@caltech.edu'], ['Emile Anand', 'eanand@caltech.edu'], ['Nicole AimeeAndrade', 'nandrade@caltech.edu'], ['Juan Arvelo', 'jarvelo@caltech.edu'], ['Abdullah Ateyeh', 'aateyeh@caltech.edu'], ['AngelAvelar Menendez', 'aavelarm@caltech.edu'], ['Reid Banciella', 'rbanciel@caltech.edu'], ['Frederik Barba', 'fbarba@caltech.edu'], ['Sarah Barrett', 'sbarrett@caltech.edu'], ['Chase Blagden', 'cblagden@caltech.edu'], ['Chase Blanchette', 'chase@caltech.edu'], ['Eve Blank', 'eblank@caltech.edu'], ['Mihir Borkar', 'mborkar@caltech.edu'], ['Celine Boucher', 'cboucher@caltech.edu'], ['James Bowden', 'jbowden@caltech.edu'], ['Max Bricken', 'mbricken@caltech.edu'], ['Matticus Brown', 'msbrown@caltech.edu'], ['Juliane Budde', 'jbudde@caltech.edu'], ['Joseph Cachaldora', 'jcachald@caltech.edu'], ['Ali Cataltepe', 'catal